# Etiquetado de imagenes para el entrenamiento del Modelo basado en YoloV8

## Definiendo funciones utiles necesarias

In [1]:
# Definiendo la clase utils para usar funciones genericas
import numpy as np
import cv2 as cv

class Utils:

    @staticmethod
    def filterKernel():
        return np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])


## Creación de las opciones básicas que usa el programa


In [2]:
import tkinter as tk
from tkinter import *
from tkinter import filedialog, ttk

class OptionsHough:
    def __init__(self):
        self.minRadius = "25"
        self.maxRadius = "45"
        self.param1 = "80"
        self.param2 = "35"
        self.mediumCellSize = "180"
        self.scale = "0.59"
        self.cameraHeight = "10"
        self.diluent = "0"        

    def return_options(self):
        return(int(self.minRadius),
                                       int(self.maxRadius),
                                       int(self.param1),
                                       int(self.param2),
                                       int(self.mediumCellSize),
                                       float(self.scale),
                                       int(self.cameraHeight),
                                       int(self.diluent)
                                       )

## Definición de la clase FrameVideo

In [3]:
class FrameVideo():

    def __init__(self, img, number_frame):
        self.circles_opencv = []
        self.number_frame = number_frame
        self.frame = img
        
    @staticmethod
    def analyzeFrame(frame_and_options):
        frame, options = frame_and_options
        filterKernel = Utils.filterKernel()
        (minRadius, maxRadius, _, _, _, _, _, _) = options


        height, width, depth = frame.frame.shape
        w = width + 50
        h = height + 50

        image = cv.cvtColor(frame.frame, cv.COLOR_BGR2GRAY)
        cv.normalize(image, image, 0, 255, cv.NORM_MINMAX)
        image = cv.filter2D(image, -1, filterKernel)
        img_border = cv.copyMakeBorder(image, 50, 50, 50, 50, cv.BORDER_REFLECT)
        circles = cv.HoughCircles(img_border, cv.HOUGH_GRADIENT, 1, 25, param1=80, param2=35, minRadius=minRadius,
                                  maxRadius=maxRadius)
        circles = np.uint16(np.around(circles))
        # I choose the circles that are inside the image
        circles2 = []
        for (x, y, r) in circles[0, :]:
            if ((x >= 50) and x <= w and (y >= 50) and y <= h):
                mask = np.zeros((height, width), np.uint8)
                cv.circle(mask, (x, y), r, 255, -1)
                points = np.transpose(np.where(mask == 255))
                sum = 0
                for (a, b) in points:
                    sum = sum + img_border[a][b]
                # suma = suma + img_border[a][b][0] + img_border[a][b][1] + img_border[a][b][2] # Aqui mejor poner algo como que > 50% pixeles tienen alta intensidad
                if (sum <= 128 * 0.6 * len(points)):
                    circles2.append([x - 50, y - 50, r])

        frame.circles_opencv = circles2
        frame.frame=image
        return frame

## Definición clase video Analyzer

In [4]:
from multiprocessing import Pool
import os

class VideoAnalyzer:

    def __init__(self, file_path):
        self.file_path = file_path
        i = self.file_path.rindex("\\")
        self.outputVideo = file_path[:i] + "\\Output\\" + file_path[i + 1:]
        os.makedirs(file_path[:i] + "\\Output\\", exist_ok=True)
        self.frames = []

    def videoSpermDetection(self, options):
        self.frames = [] #Reiniciamos por si acaso
        video = self.file_path
        outputVideo = self.outputVideo
        output_dir_images = os.path.join(os.path.dirname(outputVideo), "images")
        output_dir_labels = os.path.join(os.path.dirname(outputVideo), "labels")
        os.makedirs(output_dir_images, exist_ok=True)
        os.makedirs(output_dir_labels, exist_ok=True)

        vidcap = cv.VideoCapture(video)
        success, image = vidcap.read()
        frame_number = 0

        while success:
            f = FrameVideo(image, frame_number)
            self.frames.append(f)
            success, image = vidcap.read()
            frame_number += 1

        vidcap.release()

        jobs = [(frame, options) for frame in self.frames]
        with Pool() as p:
            self.frames = p.map(FrameVideo.analyzeFrame, jobs)
            p.close()

        for frame in self.frames:
            
            # Guardar la imagen con el cuadrado dibujado
            image_filename = f"{output_dir_images}/frame_{frame.number_frame}.jpg"
            cv.imwrite(image_filename, frame.frame)

            # Guardar las etiquetas YOLO en un archivo
            label_filename = f"{output_dir_labels}/frame_{frame.number_frame}.txt"
            with open(label_filename, "w") as f:
                for circle in frame.circles_opencv:
                    x, y, r = circle
                    # Normalizar las coordenadas X y Y y calcular el ancho y alto del cuadro
                    img_width = frame.frame.shape[1]
                    img_height = frame.frame.shape[0]
                    x_center = x / img_width
                    y_center = y / img_height
                    box_width = (2 * r) / img_width
                    box_height = (2 * r) / img_height
                    f.write(f"0 {x_center:.3f} {y_center:.3f} {box_width:.3f} {box_height:.3f}\n")

In [29]:
video = VideoAnalyzer("D:\OneDrive - Universidad de La Rioja\RepositoriosPersonales\TrabajosMaestriaCienciadeDatos\Proyecto_TFM\Supplementary_Files\Supplementary_Files\Suppl_1.avi")
options = OptionsHough()
video.videoSpermDetection(options.return_options())